# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [18]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer, OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

import warnings
warnings.filterwarnings("ignore")

In [3]:
spark = SparkSession.builder.appName('case').getOrCreate()

In [4]:
data = spark.read.csv('customer_churn.csv', header=True, inferSchema=True)

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [8]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [10]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [13]:
assembler = VectorAssembler(inputCols=['Age',
                                      'Total_Purchase',
                                      'Account_Manager',
                                      'Years',
                                      'Num_Sites'], outputCol='features')

In [14]:
output = assembler.transform(data)

In [15]:
final_data = output.select('features','churn')

In [16]:
train_churn, test_churn = final_data.randomSplit([0.8,0.2])

In [17]:
lr_churn = LogisticRegression(featuresCol='features', labelCol='churn', predictionCol='prediction')
fitted_model = lr_churn.fit(train_churn)

In [23]:
lab_and_pre = fitted_model.transform(test_churn)

In [24]:
lab_and_pre.select('prediction','churn').show()

+----------+-----+
|prediction|churn|
+----------+-----+
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    1|
|       0.0|    1|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    1|
|       1.0|    0|
|       0.0|    1|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
+----------+-----+
only showing top 20 rows



In [31]:
fitted_model.summary.areaUnderROC

0.9104012465913609

In [32]:
fitted_model.summary.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|  1.0|[0.94839870833094...|[0.72079303133929...|       0.0|
|[26.0,8939.61,0.0...|  0.0|[6.81409290944546...|[0.99890301579459...|       0.0|
|[27.0,8628.8,1.0,...|  0.0|[5.87598744773945...|[0.99720182970153...|       0.0|
|[28.0,8670.98,0.0...|  0.0|[8.22008201351031...|[0.99973087945882...|       0.0|
|[28.0,9090.43,1.0...|  0.0|[1.81543698016568...|[0.86001769949192...|       0.0|
|[28.0,11128.95,1....|  0.0|[4.52905119035248...|[0.98932429087717...|       0.0|
|[28.0,11204.23,0....|  0.0|[2.10411212980704...|[0.89130221405772...|       0.0|
|[28.0,11245.38,0....|  0.0|[4.02075048444687...|[0.98237665732249...|       0.0|
|[29.0,9378.24,0.0...|  0.0|[5.08684603680730...|[0.99386045416104...|       0.0|
|[29.0,9617.59,0

In [33]:
fitted_model.evaluate(test_churn).predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[5.10157869320793...|[0.99394969969356...|       0.0|
|[25.0,9672.03,0.0...|    0|[5.06815725686703...|[0.99374535905771...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.48086967656092...|[0.98880322631463...|       0.0|
|[29.0,8688.17,1.0...|    1|[3.01246668631828...|[0.95313416303477...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.47691041839674...|[0.97002361229145...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.32759480886796...|[0.99821686020775...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.45816767079363...|[0.98854907382557...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.43385923175663...|[0.96874612559319...|       0.0|
|[30.0,10960.52,1....|    0|[2.65403909608145...|[0.93425950203355...|       0.0|
|[32.0,6367.22,1

In [34]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [38]:
auc = churn_eval.evaluate(lab_and_pre)

In [39]:
auc

0.7502209456473707

In [40]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [41]:
test_new_customers = assembler.transform(new_customers)

In [42]:
results = fitted_model.transform(test_new_customers)

In [44]:
results.select('company','prediction').show()

+----------------+----------+
|         company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

